In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### 数据集解释：
#### emb_tb_2.csv:用户特征数据集
#### effct_tb.csv:广告点击情况数据集
#### seed_cand_tb.csv:用户类型数据集
#### 本报告主要使用广告点击情况数据，主要涉及字段如下：
#### dmp_id:营销策略编号（源数据文档未做说明，这个根据情况设定为1.对照组，2.营销策略一，3.营销策略二）
#### user_id:支付宝用户id
#### label:用户当天是否点击活动广告(0:未点击，1：点击)

In [6]:
data = pd.read_csv('./data/effect_tb.csv',header=None)
# 首次读入发现没有列名，在数据量比较大的情况下，优先获取数据的详细信息，再读入
data.columns = ['dt','user_id','label','dmp_id'] # 日期，用户id，是否点击，营销策略
data.drop(columns='dt',inplace=True)
data.head()

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2
3,1000006,0,1
4,1000006,0,3


In [7]:
# 查看数据分布  计数，均值，方差，分位数
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


# 重复值处理

In [8]:
# 检查是否存在重复行  应该是因为日期的影响，导致会有重复的数据。我的目的是看哪个营销策略会吸引更多的用户，看的是不同的用户，相同的用户在不同的时间点击多次可以排除
sum(data.duplicated())

12983

In [9]:
# 删除重复值
print('before_shape:',data.shape)
data = data.drop_duplicates()
print('重复行数',sum(data.duplicated()))
print('after_shape:',data.shape)

before_shape: (2645958, 3)
重复行数 0
after_shape: (2632975, 3)


In [10]:
# 没有重复项了
data[data.duplicated(keep=False)]

,user_id,label,dmp_id


# 空值处理

In [21]:
# 查看是否含有空值
data.isnull().sum()

user_id    0
label      0
dmp_id     0
dtype: int64

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   label    int64
 2   dmp_id   int64
dtypes: int64(3)
memory usage: 80.4 MB


# 异常值检验

In [30]:
# 通过透视表观察是否存在不合理的取值范围
data.pivot_table(index='dmp_id',columns='label',values='user_id',aggfunc='count',margins=True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


In [ ]:
正常，dmp_id = 1是对照组
      dmp_id = 2，3是实验组
    label = 0是未点击，label=1是点击了

# 假设检验

这里已经收集到数据了，做这个实验的目的是看这几个营销策略哪个表现得更好，表现更好的样本，和对照组是否有显著性差异

## 计算需要的最小样本量

In [32]:
# 对照组的点击率
data[data['dmp_id'] == 1]['label'].mean()

0.012551012429794775

对照组的广告点击率是12.6%，我们想要最少提高1%的广告点击率。那么我们就可以使用这个网站来计算需要的最小样本量 α = 0.05， β=0.2，统计效应80% Baseline conversion rate ：历史的点击率，Minimum Detectable Effect：我们想要提高最小的百分比，1% https://www.evanmiller.org/ab-testing/sample-size.html   计算出来的最小样本量是17460

In [35]:
data['dmp_id'].value_counts()

dmp_id
1    1905663
2     411107
3     316205
Name: count, dtype: int64

三种策略的样本数都大于最小样本数，样本数据合适

In [37]:
# 保存清洗好的数据。在使用python对数据清洗完毕之后，一定要保存清洗好的数据，防止在后续操作中对清洗好的数据操作失误，需要再清洗一遍数据，对于大样本来说非常消耗时间
data.to_csv('./data/out_put.csv',header=None)

## 提出假设，验证假设

看这几组的点击率是什么情况，点击率小于对照组的就不看了，哪个组的点击率大于对照组。

In [39]:
print('对照组：',data[data['dmp_id'] == 1]['label'].mean())
print('策略组1：',data[data['dmp_id'] == 2]['label'].mean())
print('策略组2：',data[data['dmp_id'] == 3]['label'].mean())

对照组： 0.012551012429794775
策略组1： 0.015314747742072015
策略组2： 0.026191869198779274


策略组1和策略组2都有提升。策略1组提升了0.2%，策略组2提升了1.3%。只有策略组2满足我们的大于1%的要求。所以接下来我们就需要提出假设验证这个结果是显著的

### 提出假设

- 原假设和备择假设
- 设对照组的点击率为p1，策略2组的点击率为p2
- 原假设：H0 p1>=p2  备择假设：H1  p1<p2
- 检验统计量的选择：样本属于大样本，均值和方差均未知，选择Z检验统计量

In [42]:
control_g = data[data['dmp_id'] == 1]['label'].mean()
control_sum = len(data[data['dmp_id'] == 1]['label'])
ex_g = data[data['dmp_id'] == 3]['label'].mean()
ex_sum  = len(data[data['dmp_id'] == 3]['label'])
print('对照组的点击率:',control_g,'实验组的点击率:',ex_g)

对照组的点击率: 0.012551012429794775 实验组的点击率: 0.026191869198779274


这里为什么我使用的是两个样本各自的比例，而不是合并的比例
- 使用合并的比例是基于两个样本来自于统一个总体，并且是基线比例相同。但是这是一种理想状态。
- 使用两个样本的各自比例比较合适，接近于现实

In [45]:
# 计算检验统计量的z值
z = (control_g - ex_g) / (np.sqrt((1-control_g)*control_g / control_sum + (1-ex_g)*ex_g / ex_sum))
print('检验统计量z:',z)

检验统计量z: -46.20280310569833


In [46]:
# 看显著性水平0.05对应的z分数
from  scipy.stats import norm
z_alpha = norm.ppf(0.05)
print('z_alpha:',z_alpha)

z_alpha: -1.6448536269514729


z_alpha = -1.64,检验统计量是-46。改检验是左侧单尾检验，拒绝域是{z<z_alpha}，拒绝原假设。接下来计算p值

In [50]:
p_value = 2 * (1 - norm.cdf(np.abs(z)))
p_value

0.0

p值小于0.05，和使用z检验统计量相同。所以拒绝原假设。策略2相比于对照组有显著性差异。